In [ ]:
# torch-related imports
import torch
from torch import nn, Tensor
from torch.autograd import Variable
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import DataLoaderIter
from torchvision.transforms import Compose, RandomRotation, RandomResizedCrop, ToTensor

# helper functions
from os.path import join

# full libraries
import matplotlib.pyplot as plt

# my classes
from FlickrDataset import FlickrDataset

# set random seeds
seed = 22
_ = torch.random.manual_seed(seed)

In [ ]:
flickrData = FlickrDataset(join("data", "flickr_logos_27_dataset", "flickr_logos_27_dataset_training_set_annotation.txt"),
                          join("data", "flickr_logos_27_dataset", "flickr_logos_27_dataset_images"),
                          transform=Compose([
                              RandomRotation(10),
                              RandomResizedCrop(256),
                              ToTensor()
                          ]))

In [ ]:
batch_size = 16
flickrLoader = DataLoader(flickrData,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2,
                          drop_last=True)

In [ ]:
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 4, kernel_size=3, stride=2),
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        print(self.layer1)
        self.layer2 = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=3, stride=2),
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        print(self.layer2)
        self.linear1 = nn.Sequential(
            nn.Linear(900, 27, bias=True),
            nn.ReLU()
        )
        print(self.linear1)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.linear1(out)
        return out

In [ ]:
classifier = MyClassifier()
learning_rate = 0.0001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 100
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(DataLoaderIter(flickrLoader)):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = classifier(images)
        l = loss(outputs, labels)
        l.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(flickrData)//batch_size, l.data[0]))
